In [82]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [83]:
bnf_1min = pd.read_csv("../data/gold_tv_4h.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2012]

In [84]:
bnf_1min.head()

,time,open,high,low,close,MA,Shapes,Shapes.1,RSI,datetime
0,2015-03-02T09:00:00+05:30,26600,26769,26600,26700,NaN,0,0,NaN,2015-03-02 09:00:00+05:30
1,2015-03-02T13:00:00+05:30,26709,26730,26653,26680,NaN,0,0,NaN,2015-03-02 13:00:00+05:30
2,2015-03-02T17:00:00+05:30,26684,26742,26576,26619,NaN,0,0,NaN,2015-03-02 17:00:00+05:30
3,2015-03-02T21:00:00+05:30,26623,26624,26500,26506,NaN,0,0,NaN,2015-03-02 21:00:00+05:30
4,2015-03-03T09:00:00+05:30,26500,26525,26444,26463,NaN,0,0,NaN,2015-03-03 09:00:00+05:30


In [85]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2022, 8, 4),
 datetime.date(2016, 2, 12),
 datetime.date(2021, 4, 21),
 datetime.date(2023, 5, 3),
 datetime.date(2021, 5, 19),
 datetime.date(2018, 6, 29),
 datetime.date(2020, 1, 2),
 datetime.date(2023, 2, 7),
 datetime.date(2022, 2, 16),
 datetime.date(2018, 7, 25),
 datetime.date(2022, 3, 7),
 datetime.date(2016, 6, 29),
 datetime.date(2017, 1, 13),
 datetime.date(2015, 6, 29),
 datetime.date(2016, 3, 17),
 datetime.date(2019, 11, 20),
 datetime.date(2019, 3, 22),
 datetime.date(2022, 1, 12),
 datetime.date(2022, 9, 12),
 datetime.date(2020, 6, 17),
 datetime.date(2023, 7, 13),
 datetime.date(2015, 3, 5),
 datetime.date(2021, 1, 21),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 10, 22),
 datetime.date(2022, 4, 6),
 datetime.date(2023, 10, 20),
 datetime.date(2019, 12, 4),
 datetime.date(2017, 5, 29),
 datetime.date(2022, 1, 14),
 datetime.date(2023, 9, 21),
 datetime.date(2017, 3, 1),
 datetime.date(2019, 4, 5),
 datetime.date(2018, 7, 3),
 datetime.date(2016, 

In [86]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
RPT_PCT = 0.05 # 5% RPT
SLIPPAGE_ = 0.001
LEVERAGE_ = 5

In [87]:
data = bnf_1min

In [88]:
def calculate_signals(df, ma_length, ema_length):
    df['MA'] = df['close'].rolling(ma_length).mean()
    df['EMA'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['MA'] >= df['EMA']) & 
        (df['MA'].shift(1) < df['EMA'].shift(1))
    )
    return df

In [89]:
def backtest(df, fractal_num, sl_pct):
    
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    # tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    tsl_time = None
    points = 0

    for i in range(1, len(df)):

        if df.loc[i-1, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i-1, 'close']
            long_entry_date = df.loc[i-1, 'datetime']
            long_initial_sl = long_entry_price * (1 - (sl_pct) / 100)
            long_trailing_sl = long_initial_sl

        if long_position == 1:
            if all(df.loc[i - j, 'low'] >= df.loc[i - fractal_num, 'low'] for j in range(0, ((fractal_num * 2) + 1))):
                # Update Trailing SL
                long_trailing_sl = df.loc[i - fractal_num, 'low']
                tsl_time = df.loc[i - fractal_num, 'datetime']

            if df.loc[i, 'low'] <= long_initial_sl:
                # Initial SL Hit
                long_position = 0
                long_exit_price = long_initial_sl
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'Initial SL'

            elif df.loc[i, 'close'] <= long_trailing_sl:
                # Trailing SL Hit
                long_position = 0
                long_exit_price = df.loc[i, 'close']
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'Trailing SL'

            elif df.loc[i, 'MA'] < df.loc[i, 'EMA']:
                # MA Cross
                long_position = 0
                long_exit_price = df.loc[i, 'close']
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'MA Bear Cross'

        if points:
            slippage = (long_entry_price + long_exit_price) * SLIPPAGE_
            qty = RPT_PCT * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl)
            trade = {
                "Entry Date": long_entry_date.date(),
                "Entry Time": long_entry_date.time(),
                "Entry Price": long_entry_price,
                "Initial SL": long_initial_sl,
                'Final SL': long_trailing_sl,
                'TSL Time': tsl_time,
                "Exit Date": long_exit_time.date(),
                "Exit Time": long_exit_time.time(),
                "Exit Price": long_exit_price,
                "Points Captured": points,
                "Slippages": slippage,
                "Points w cs": points - slippage,
                "PnL": (points - slippage) * qty,
                "Remarks": remark,
                'Qty': qty,
                "ROI%": ((points - slippage) * qty / PORTFOLIO_VALUE) * 100,
                "Trade Year": long_entry_date.year,
                "Trade Month": long_entry_date.month,
            }
            # print(trade)
            long_trades.append(trade)
            points = 0
            remark = ''

    tradebook = pd.DataFrame(long_trades)
    return tradebook

In [90]:
MA_LENGTH = 9
EMA_LENGTH = 30
FRACTAL_LENGTH = 5
SL_PCT = 1

df = calculate_signals(data, MA_LENGTH, EMA_LENGTH)
tb = backtest(df, FRACTAL_LENGTH, SL_PCT)

In [91]:
tb

,Entry Date,Entry Time,Entry Price,Initial SL,Final SL,TSL Time,Exit Date,Exit Time,Exit Price,Points Captured,Slippages,Points w cs,PnL,Remarks,Qty,ROI%,Trade Year,Trade Month
0,2015-03-20,17:00:00,26108,25846.9200,25846.9200,NaT,2015-04-01,09:00:00,26320.0000,212.0000,52.4280,159.5720,30559.9816,MA Bear Cross,191.5122,3.0560,2015,3
1,2015-04-01,17:00:00,26620,26353.8000,26225.0000,2015-03-31 21:00:00+05:30,2015-04-10,17:00:00,26758.0000,138.0000,53.3780,84.6220,15894.4403,MA Bear Cross,187.8287,1.5894,2015,4
2,2015-04-13,17:00:00,26669,26402.3100,26402.3100,2015-03-31 21:00:00+05:30,2015-04-14,17:00:00,26402.3100,-266.6900,53.0713,-319.7613,-59950.0000,Initial SL,187.4836,-5.9950,2015,4
3,2015-04-17,13:00:00,26745,26477.5500,26606.0000,2015-04-22 21:00:00+05:30,2015-04-24,17:00:00,26674.0000,-71.0000,53.4190,-124.4190,-23260.2356,MA Bear Cross,186.9508,-2.3260,2015,4
4,2015-04-24,21:00:00,26720,26452.8000,26630.0000,2015-04-24 21:00:00+05:30,2015-05-01,17:00:00,26599.0000,-121.0000,53.3190,-174.3190,-32619.5734,Trailing SL,187.1257,-3.2620,2015,4
5,2015-05-07,13:00:00,26967,26697.3300,26697.3300,2015-04-24 21:00:00+05:30,2015-05-11,17:00:00,26802.0000,-165.0000,53.7690,-218.7690,-40562.3540,MA Bear Cross,185.4118,-4.0562,2015,5
6,2015-05-12,17:00:00,27145,26873.5500,27246.0000,2015-05-15 13:00:00+05:30,2015-05-20,09:00:00,27227.0000,82.0000,54.3720,27.6280,5088.9667,Trailing SL,184.1960,0.5089,2015,5
7,2015-06-03,13:00:00,27140,26868.6000,26868.6000,2015-05-15 13:00:00+05:30,2015-06-04,17:00:00,26868.6000,-271.4000,54.0086,-325.4086,-59950.0000,Initial SL,184.2299,-5.9950,2015,6
8,2015-06-11,09:00:00,26918,26648.8200,26790.0000,2015-06-11 17:00:00+05:30,2015-06-15,09:00:00,26932.0000,14.0000,53.8500,-39.8500,-7402.1101,MA Bear Cross,185.7493,-0.7402,2015,6
9,2015-06-15,17:00:00,26998,26728.0200,26803.0000,2015-06-17 17:00:00+05:30,2015-06-22,17:00:00,26728.0200,-269.9800,53.7260,-323.7060,-59950.0000,Initial SL,185.1989,-5.9950,2015,6


In [92]:
tb['ROI%'].sum()

169.21121818263987

In [93]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2015, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2015, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [69]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2015,-32.7295,19,21.0526,8.3583,-4.4108,-37.3749,-0.8757,...
2016,77.9353,18,38.8889,18.1047,-4.4362,-24.5098,3.1798,...
2017,22.9008,13,46.1538,8.4390,-3.9619,-16.7799,1.3648,...
2018,-14.3789,17,35.2941,6.0969,-4.6327,-39.0128,-0.3686,...
2019,-1.7837,19,26.3158,14.0347,-5.1398,-39.1671,-0.0455,...
2020,7.1627,21,33.3333,9.4462,-4.2115,-35.0910,0.2041,...
2021,-4.2747,19,36.8421,6.7713,-4.3062,-27.6437,-0.1546,...
2022,37.4222,22,31.8182,15.5478,-4.7608,-42.2996,0.8847,...
2023,47.5209,15,53.3333,8.5325,-2.9627,-13.2096,3.5974,...
2024,29.4362,13,38.4615,13.5034,-4.7601,-14.3413,2.0525,...


In [80]:
# MA_LENGTH = 9
# EMA_LENGTH = 30
# FRACTAL_LENGTH = 5
# SL_PCT = 1

# df = calculate_signals(data, MA_LENGTH, EMA_LENGTH)
# tb = backtest(df, FRACTAL_LENGTH, SL_PCT)

sl_range = [0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5]
stats_dictionary = {}

for i in range(8, 33, 2):
    for j in range(5, 81, 5):
        for k in range(1, 6):
            for sl in sl_range:
                if i < j and (j - i) >= 6:
                    df = calculate_signals(data, i, j)
                    tb = backtest(df, k, sl)
                    variation = f'{i}, {j}, {k}, {sl}'
                    print(variation)
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        if x > 6:
                            print(y.to_string())
                            stats_dictionary[x] = y

8, 15, 1, 0.5
8, 15, 1, 0.75
8, 15, 1, 1
8, 15, 1, 1.25
8, 15, 1, 1.5
8, 15, 1, 1.75
8, 15, 1, 2
8, 15, 1, 2.25
8, 15, 1, 2.5
8, 15, 2, 0.5
8, 15, 2, 0.75
8, 15, 2, 1
8, 15, 2, 1.25
8, 15, 2, 1.5
8, 15, 2, 1.75
8, 15, 2, 2
8, 15, 2, 2.25
8, 15, 2, 2.5
8, 15, 3, 0.5
8, 15, 3, 0.75
8, 15, 3, 1
8, 15, 3, 1.25
8, 15, 3, 1.5
8, 15, 3, 1.75
8, 15, 3, 2
8, 15, 3, 2.25
8, 15, 3, 2.5
8, 15, 4, 0.5
8, 15, 4, 0.75
8, 15, 4, 1
8, 15, 4, 1.25
8, 15, 4, 1.5
8, 15, 4, 1.75
8, 15, 4, 2
8, 15, 4, 2.25
8, 15, 4, 2.5
8, 15, 5, 0.5
8, 15, 5, 0.75
8, 15, 5, 1
8, 15, 5, 1.25
8, 15, 5, 1.5
8, 15, 5, 1.75
8, 15, 5, 2
8, 15, 5, 2.25
8, 15, 5, 2.5
8, 20, 1, 0.5
8, 20, 1, 0.75
8, 20, 1, 1
8, 20, 1, 1.25
8, 20, 1, 1.5
8, 20, 1, 1.75
8, 20, 1, 2
8, 20, 1, 2.25
8, 20, 1, 2.5
8, 20, 2, 0.5
8, 20, 2, 0.75
8, 20, 2, 1
8, 20, 2, 1.25
8, 20, 2, 1.5
8, 20, 2, 1.75
8, 20, 2, 2
8, 20, 2, 2.25
8, 20, 2, 2.5
8, 20, 3, 0.5
8, 20, 3, 0.75
8, 20, 3, 1
8, 20, 3, 1.25
8, 20, 3, 1.5
8, 20, 3, 1.75
8, 20, 3, 2
8, 20, 3, 2.25
8, 20,

In [81]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{9.859810381564069:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2015      11.7586            9  44.4444                5.8020   
 2016      69.3937            9  44.4444               22.3290   
 2017      18.6702            8  25.0000               18.3489   
 2018      13.1269           10  50.0000                6.0420   
 2019      31.0399           13  23.0769               25.5246   
 2020      19.4563           11  36.3636               13.3017   
 2021      17.9513            9  44.4444                8.7372   
 2022      70.5093           13  53.8462               12.9251   
 2023      25.4779            9  55.5556                8.0686   
 2024      -1.7457            8  37.5000                7.7057   
 Overall  275.6383           99  41.4141               12.1487   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio        Variation  
 2015                -2.2899      -6.8418       1.7186  14, 55, 5, 1.25  
 2016                -3.9844     -19.92